In [1]:
!pip install openai
!pip install langchain
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.7 MB/s eta 0:00:00


In [2]:
import os

In [3]:
from langchain.chat_models import ChatOpenAI

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key_team0')

In [4]:
chat_model = ChatOpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# 음식점 소유자로서, 사용자(user) comment 답변하는 role 부여

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Act as a kind and excellent restaurant owner. Respond to the user-written comment. For negative comments, provide a detailed apology and mention specific areas for improvement. For positive comments, express your gratitude in detail."),
    ("human", "{input_text}"),
])

chain = chat_prompt_template | chat_model | StrOutputParser()
# LangChain Expression Language (LCEL)

def langchain_llm(input_text):
    output = chain.invoke({"input_text": input_text})
    return output

In [7]:
# 답변 생성
def generate_reply(input_text):
    output = langchain_llm(input_text)
    return output

In [8]:
#필요 데이터 다운로드
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarrr_sample_submit.txt",
    filename="tarrr_sample_submit.txt",
)

('tarrr_sample_submit.txt', <http.client.HTTPMessage at 0x7c863ad82e60>)

In [9]:
import pandas as pd

path = '/content/tarrr_sample_submit.txt'

df = pd.read_csv(path, delimiter='\t')
df.head(3)

,id,comment,reply
0,1,완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌,의견 감사합니다.
1,2,맛있긴 한데 양이 너무 적어서 좀... ㅠ,의견 감사합니다.
2,3,완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨,의견 감사합니다.


In [10]:
total = len(df)

for index, row in df.iterrows():
    comment = row['comment']
    reply = generate_reply(comment)

    print(f'[{index+1}]/[{total}]')
    print("comment: ", comment)
    print("reply: ", reply)
    print("--"*30)

    if index > 4 :
        break

[1]/[100]
comment:  완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌
reply:  감사합니다! 손님의 좋은 평가에 감사드립니다. 우리 레스토랑이 손님의 취향에 부합하고 가격과 위치도 만족스러우셨다니 정말 기쁩니다. 항상 최상의 서비스와 맛있는 요리로 손님들을 모시고자 최선을 다하겠습니다. 다음에도 손님을 맞이할 기회가 생긴다면 더욱 특별한 경험을 제공해드릴 수 있도록 노력하겠습니다. 다시 한 번 감사드립니다! 🌟
------------------------------------------------------------
[2]/[100]
comment:  맛있긴 한데 양이 너무 적어서 좀... ㅠ
reply:  저희 레스토랑을 이용해 주셔서 감사합니다. 양이 부족했다니 죄송합니다. 고객님의 소중한 의견을 반영하여 음식의 양을 더 풍성하게 하도록 노력하겠습니다. 다음에 또 방문해 주시면 보다 나은 서비스를 제공해 드릴 수 있도록 더 노력하겠습니다. 불편을 끼쳐드려 죄송합니다. 이용해 주셔서 진심으로 감사드립니다.
------------------------------------------------------------
[3]/[100]
comment:  완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨
reply:  감사합니다! 손님의 칭찬에 감사드립니다. 우리 레스토랑의 분위기를 즐겁게 즐겨주셔서 기쁩니다. 항상 최선을 다해 맛있는 음식과 아늑한 분위기를 제공할 수 있도록 노력하겠습니다. 다음에 또 방문해주시면 더 많은 즐거움을 느끼실 수 있도록 노력하겠습니다. 감사합니다!
------------------------------------------------------------
[4]/[100]
comment:  한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.
reply:  감사합니다! 한국의 전통 음식을 잘 표현할 수 있어서 기쁩니다. 고객님께서 향토음식의 정취를 느끼셨다니 저희 레스토랑의 목표를 